## 1

In [ ]:
import gdown

url = 'https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM'
output = 'GoogleNews-vectors-negative300.bin.gz'
gdown.download(url, output, quiet=False)

In [ ]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin.gz', binary=True)

print(model['United_States'])

## 2

In [ ]:
print(model.similarity('United_States', 'U.S.'))

## 3

In [ ]:
model.most_similar('United_States', topn=10)

In [ ]:
model.most_similar(positive=['United_States'], topn=10)

## 4

In [ ]:
vec = model['Spain'] - model['Madrid'] + model['Athens'] 
model.most_similar(positive=['Spain', 'Athens'], negative=['Madrid'], topn=10)#結果違う

In [ ]:
model.most_similar(vec, topn=10)#結果違う

## 5

In [ ]:
!wget http://download.tensorflow.org/data/questions-words.txt

In [ ]:
with open('./questions-words.txt', 'r') as fr:
    with open('./ans_of_5.txt', 'w') as fw:
        analogy_num=''
        for line in fr:
          line = line.split()
          if line[0] == ':':
            analogy_num=line[1]
          else:
            vec = model[line[1]] - model[line[0]] + model[line[2]]
            #add_word, similarity = model.most_similar(vec, topn=1)[0]
            add_word, similarity = model.most_similar(positive=[line[1], line[2]], negative=[line[0]], topn=1)[0]
            line=' '.join(line)
            fw.write(analogy_num + ' ' + line + ' ' + add_word + ' ' + str(similarity) + '\n')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 6

In [ ]:
sem_cnt = 0
sem_cor = 0
syn_cnt = 0
syn_cor = 0

with open('./questions-words-add.txt', 'r') as f:
  for line in f:
    line = line.split()
    if not line[0].startswith('gram'):
      sem_cnt += 1
      if line[4] == line[5]:
        sem_cor += 1
    else:
      syn_cnt += 1
      if line[4] == line[5]:
        syn_cor += 1

print(f'意味的アナロジー正解率: {sem_cor/sem_cnt:.3f}')
print(f'文法的アナロジー正解率: {syn_cor/syn_cnt:.3f}') 

## 7

In [ ]:
!wget http://www.gabrilovich.com/resources/data/wordsim353/wordsim353.zip
!unzip wordsim353.zip

In [ ]:
import numpy as np
from scipy.stats import spearmanr

ws353 = []
with open('./combined.csv', 'r') as f:
  next(f)
  for line in f:  # 1行ずつ読込み、単語ベクトルと類似度を計算
    line = [s.strip() for s in line.split(',')]
    line.append(model.similarity(line[0], line[1]))
    ws353.append(line)


# スピアマン相関係数の計算
human = np.array(ws353).T[2]
w2v = np.array(ws353).T[3]
correlation, pvalue = spearmanr(human, w2v)

print(f'スピアマン相関係数: {correlation:.3f}')

## 8

In [ ]:
# 国名の取得
countries = set()
with open('./ans_of_5.txt') as f:
  for line in f:
    line = line.split()
    if line[0] in ['capital-common-countries', 'capital-world']:
      countries.add(line[2])
    elif line[0] in ['currency', 'gram6-nationality-adjective']:
      countries.add(line[1])
countries = list(countries)

# 単語ベクトルの取得
countries_vec = [model[country] for country in countries]

from sklearn.cluster import KMeans

# k-meansクラスタリング
kmeans = KMeans(n_clusters=5)
kmeans.fit(countries_vec)
for i in range(5):
    cluster = np.where(kmeans.labels_ == i)[0]
    print('cluster', i)
    print(', '.join([countries[k] for k in cluster]))

## 9

In [ ]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage

plt.figure(figsize=(15, 5))
Z = linkage(countries_vec, method='ward')
dendrogram(Z, labels=countries)
plt.show()

## 10

In [ ]:
 !pip install bhtsne

In [ ]:
import bhtsne

embedded = bhtsne.tsne(np.array(countries_vec).astype(np.float64), dimensions=2, rand_seed=123)
plt.figure(figsize=(10, 10))
plt.scatter(np.array(embedded).T[0], np.array(embedded).T[1])
for (x, y), name in zip(embedded, countries):#plt.annotate(name, (x, y))によって、データの名前nameを座標(x, y)の位置にアノテーションとして表示します。この処理により、各データポイントに対応する名前がプロット上に表示されます。
    plt.annotate(name, (x, y))
plt.show()